# Signature Optimal Stopping applied to mean-reverting spreads

Advanced notebook using the Rust `signature_optimal_stopping` crate (truncated signatures + ridge regression).

Goals:
- simulate mean-reverting spreads (discrete OU)
- build trajectory dataset and corresponding rewards
- train the model in Rust (via CLI) or via Python bindings
- evaluate the stopping rule on test trajectories


In [ ]:
# Python imports for simulation and CLI interaction
import json
import subprocess
import tempfile
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os


## 1) Simulate a mean-reverting spread (discrete OU)
Dynamics: x_{t+1} = x_t + kappa*(mu - x_t)*dt + sigma*sqrt(dt)*eps

In [ ]:
def simulate_ou(n_steps=60, mu=0.0, kappa=0.1, sigma=0.5, dt=1.0, x0=0.0):
    x = x0
    traj = [x]
    for _ in range(n_steps-1):
        eps = np.random.normal()
        x = x + kappa*(mu - x)*dt + sigma*np.sqrt(dt)*eps
        traj.append(x)
    return np.array(traj)

# demo plot
np.random.seed(42)
s = simulate_ou(100, mu=0.0, kappa=0.2, sigma=1.0)
plt.plot(s); plt.title('OU demo'); plt.show()


## 2) Build dataset: trajectories + reward
We simulate trajectories and produce a target reward representing the value if stopped now (example: future average or final value).

In [ ]:
def build_samples_from_ou(n_samples=200, n_steps=30, H=1, cost=0.0):
    samples = []
    for _ in range(n_samples):
        traj = simulate_ou(n_steps)
        prefix = [[float(v)] for v in traj[:n_steps]]
        if n_steps + H < len(traj):
            future = traj[n_steps:n_steps+H].mean()
        else:
            future = traj[-1]
        reward = float(future - traj[-1] - cost)
        samples.append({'traj': prefix, 'reward': reward})
    return samples

samples = build_samples_from_ou(150, 20)
print('samples built:', len(samples))


## 3) Train the Rust model via CLI
Write the dataset to JSON, call the trainer binary, and read the produced weights.

In [ ]:
def train_via_cli(samples, trunc=2, ridge=1e-3, bin_path='./target/release/trainer'):
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix='.json')
    json.dump({'params': {'truncation': trunc, 'ridge': ridge}, 'samples': samples}, tmp)
    tmp.close()
    out_weights = tmp.name + '.weights.json'
    cmd = [bin_path, 'train', '--input', tmp.name, '--output', out_weights]
    try:
        res = subprocess.run(cmd, capture_output=True, check=True, text=True)
        print('trainer stdout:', res.stdout)
    except Exception as e:
        print('Trainer call failed (ensure binary exists):', e)
        return None
    if os.path.exists(out_weights):
        with open(out_weights, 'r') as f:
            w = json.load(f)
        return w
    return None

# Example call (requires building the crate)
# weights = train_via_cli(samples, trunc=2, ridge=1e-3, bin_path='./target/release/trainer')


## 4) Evaluate stopping rule (if weights are available)
Compute truncated signature in Python using the same simple discrete formula and dot with weights.

In [ ]:
def evaluate_rule_on_samples(weights, samples, trunc=2):
    results = []
    for s in samples:
        traj = s['traj']
        # compute features as in Rust
        d = len(traj[0])
        inc = []
        for i in range(1, len(traj)):
            inc.append([traj[i][k] - traj[i-1][k] for k in range(d)])
        feat = []
        if trunc >= 1:
            for k in range(d):
                feat.append(sum(x[k] for x in inc))
        if trunc >= 2:
            for i in range(d):
                for j in range(d):
                    ssum = 0.0
                    for a in range(len(inc)):
                        for b in range(a+1, len(inc)):
                            ssum += inc[a][i] * inc[b][j]
                    feat.append(ssum)
        import numpy as _np
        w = _np.array(weights)
        x = _np.array(feat)
        score = float(_np.dot(w, x)) if w.size == x.size else None
        results.append({'score': score, 'reward': s['reward']})
    return results

print('Notebook ready. Build the Rust crate and run the trainer to obtain weights or expose PyO3 bindings for direct calls.')


## Integration notes
- For direct Python usage, write a PyO3 wrapper exposing `SignatureStopper::train`, `score`, and `should_stop`.
- Alternatively use the provided trainer binary to train and produce weights.json which can be read by Python.
- The approach follows the paper's idea: truncated signature features and regression to approximate continuation values. The paper includes more advanced ideas (sampling, penalties, bagging) that can be implemented on top of this base in Rust.